In [ ]:
!pip install razdel

In [ ]:
import re
import seaborn as sns
import numpy as np

from scipy import spatial
from matplotlib import pyplot as plt

import razdel

In [ ]:
import pandas as pd

In [ ]:
tails = pd.read_excel('Tails.xlsx')

In [ ]:
tails

,Name,Kunadinsky,Russian
0,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,"Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
1,Куманды калыктыҥ тазылы. Кип-Чӧрчек / Корни ку...,По Кип-Чӧрчекти кӧргенде ирткан тушта чер чайа...,"Согласно легенде, в древние времена духи приро..."
2,Чер-чарык чайалганы / Легенда о создании мира,Ӱӱрде ирткен качандаа кудай учы-тӱби чок талай...,Давным-давно Бог создал бескрайнее море и небо...
3,Апшыйач ла куртыйач / Старик и старушка,"Пир тушта, качандаа чыш черде, айры чагазында,...",Когда-то в лесном краю на берегу реки жили люд...
4,Альчи ла кийик / Альчи и олень,"Тӱндӱк черде, ньаан Умар айрыныҥ чагазында кий...","На севере, на берегу большой реки Обь жил наро..."
5,Аҥчы ла Челбеен / Охотник и Челбеен,"Аба чыштыҥ аразында пуруна, пуруна чатында кар...","Давным-давно в таёжном краю, у подножья чёрной..."
6,Игерниҥ тӧртиги / Собачий хлеб,Качан – качандаа Алтайда ньаан калык чаткан. О...,Когда-то на Алтае жил большой народ. Люди на э...
7,Тöреен черим чÿрегимде / Родная земля в сердце,"Кöк ээниг сында, ээс тагларныҥ кийниндеҥ таҥ ч...","На синей широкой гриве, за высокими горами на ..."


In [ ]:
name = tails['Name'].tolist()
kum = tails['Kunadinsky'].tolist()
rus = tails['Russian'].tolist()

In [ ]:
for i in range(len(name)):
  print(name[i], '--', '\n', kum[i], '\n\n', rus[i], '\n', '-'*40)

Куу калыктыҥ чӧрчеги / Сказка о лебедином народе -- 
 Качан, качандаа чер сыртында пир ньаан калык чаткан. Анар пойыны кууныҥ калыгы деп аданар. Алдындаҥ пери анаҥ аназы куу полгон деп анар пӱткенер. Анар чакшы кӧӧнилиг, улузыраш калык полгон. Коштыйа анаҥ ла чатчын калык куу калыкты тоога салган, кӱндӱлен. Ӧзере садыжып, положып ӧштиглердин ӧзере аданыны алыжып чуртарыны абыр тутып чатканар. Че по канчеп таа чыллар, шактар ӧте перерде, куу калык пойыныҥ чайалганыны тыҥ улугзыранып, пактанып, тазаҥнап чӱре пердилер. «- Пис чараш куудаҥ шыккан калык!» - деп, айлантыра чаткан калыкка айтканар. «- По куш тыҥ ээс кӧк тегриде кудайга чагын ӱчип чӱрчит. Пистиҥ калык угала даа тыҥ, аҥ учын парчызы писти кӱндӱлерге керек». Ийде пойыныҥ чайалганыны ӧрӧ кӧдӱрип, аалында чакшы иштенген мелерини кичигден, кичигден уҥдыйа пенер. Коштыйа чатчын калыктарла ӧштеже пердилер, ӧзере сатыштанды токтатсалдылар, ус иштейтен мелерини кеде эдип, чакшы алыптыг чӧрчӧктерини ийтенини уҥдып салдылар. Канчеп тьаа 

In [ ]:
class corpora():

  def __init__(self, text_kum, text_ru, name: str):
    self.text_kum = text_kum
    self.text_ru = text_ru
    self.name = name
    self.splt = self.spliting()

  def spliting(self):
    kum = re.sub('\n', ' ', self.text_kum)
    rus = re.sub('\n', ' ', self.text_ru)
    self.sent_kum = list(x.text for x in razdel.sentenize(kum))
    self.sent_rus = list(x.text for x in razdel.sentenize(rus))
    print(f'С пропусками kum: {len(self.sent_kum)} ru: {len(self.sent_rus)}')

    if len(self.sent_kum) > len(self.sent_rus):
      razn = len(self.sent_kum) - len(self.sent_rus)
      for i in range(razn):
        self.sent_rus.append('No data')
    elif len(self.sent_kum) < len(self.sent_rus):
      diff = len(self.sent_rus) - len(self.sent_kum)
      for n in diff:
        self.sent_kum.append('No data')
    print(f'Без пропусков kum: {len(self.sent_kum)} ru: {len(self.sent_rus)}')

    return self.name, self.sent_kum, self.sent_rus


  def create_corpora(self):
    data_list = []
    for i in range(len(self.splt[1])):
      row = {'Name': self.splt[0], 'Sent_kum': self.splt[1][i], 'Sent_rus': self.splt[2][i], 'Text_kum': self.text_kum, 'Text_rus': self.text_ru}
      data_list.append(row)

    df = pd.DataFrame(data_list)

    return df

In [ ]:
corp = corpora(kum[0], rus[0], name[0])
a = corp.create_corpora()

С пропусками kum: 23 ru: 22
Без пропусков kum: 23 ru: 23


In [ ]:
a

,Name,Sent_kum,Sent_rus,Text_kum,Text_rus
0,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,"Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ.,"Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
1,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,Анар пойыны кууныҥ калыгы деп аданар.,Они называли себя лебединым народом.,"Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
2,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,Алдындаҥ пери анаҥ аназы куу полгон деп анар п...,"Они верили, что их праматерью была лебедь.","Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
3,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,"Анар чакшы кӧӧнилиг, улузыраш калык полгон.","Они были гордыми, с хорошими нравами.","Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
4,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,Коштыйа анаҥ ла чатчын калык куу калыкты тоога...,"Живущий рядом народ считался с ними, они уважа...","Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
5,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,"Ӧзере садыжып, положып ӧштиглердин ӧзере аданы...",Но со временем лебединый народ возгордился сво...,"Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
6,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,"Че по канчеп таа чыллар, шактар ӧте перерде, к...",«Мы произошли от прекрасного лебедя!» – говори...,"Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
7,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,"«- Пис чараш куудаҥ шыккан калык!» - деп, айла...","«Эта птица летает в небе, близко к богу.","Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
8,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,«- По куш тыҥ ээс кӧк тегриде кудайга чагын ӱч...,"Наш народ мощный, все должны нас почитать!»","Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
9,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,"Пистиҥ калык угала даа тыҥ, аҥ учын парчызы пи...","И так возвышая своё происхождение, постепенно,...","Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...


In [ ]:
class second_corpora():

  def __init__(self, text_kum: list, text_ru: list, name: list):
    self.text_kum = text_kum
    self.text_ru = text_ru
    self.name = name
    self.splt = self.spliting()

  def spliting(self):
    new_line = '\n'
    for tail in range(len(self.text_kum)):
      kum = re.sub('\n', ' ', self.text_kum[tail])
      rus = re.sub('\n', ' ', self.text_ru[tail])
      sent_kum = list(x.text for x in razdel.sentenize(kum))#Одна сказка
      sent_rus = list(x.text for x in razdel.sentenize(rus))#Одна сказка
      print(f'{name}{new_line}С пропусками kum: {len(self.sent_kum)} ru: {len(self.sent_rus)}')

      if len(sent_kum) > len(sent_rus):
        razn = len(sent_kum) - len(sent_rus)
        for i in range(razn):
          sent_rus.append('No data')
      elif len(sent_kum) < len(sent_rus):
        diff = len(sent_rus) - len(sent_kum)
        for n in diff:
          sent_kum.append('No data')
      print(f'{name}{new_line}Без пропусков kum: {len(sent_kum)} ru: {len(sent_rus)}{new_line}')

    return self.name, self.sent_kum, self.sent_rus


  def create_corpora(self):
    data_list = []
    for i in range(len(self.splt[1])):
      row = {'Name': self.splt[0], 'Sent_kum': self.splt[1][i], 'Sent_rus': self.splt[2][i], 'Text_kum': self.text_kum, 'Text_rus': self.text_ru}
      data_list.append(row)

    df = pd.DataFrame(data_list)

    return df

In [ ]:
class third_corpora():

  def __init__(self, text_kum: list, text_ru: list, name: list):
    self.text_kum = text_kum
    self.text_ru = text_ru
    self.name = name
    # self.splt = self.spliting()

  def spliting(self, name, tail_kum, tail_ru):
    kum = re.sub('\n', ' ', tail_kum)
    rus = re.sub('\n', ' ', tail_ru)
    sent_kum = list(x.text for x in razdel.sentenize(kum))
    sent_rus = list(x.text for x in razdel.sentenize(rus))
    # print(f'С пропусками kum: {len(sent_kum)} ru: {len(sent_rus)}')

    #ОБРАБОТКА РАЗНОГО КОЛ-ВА ПРЕДЛОЖЕНИЙ. ДОБАВЛЕНИЕ ПУСТЫХ ПРЕДЛОЖЕНИЙ ДЛЯ ПОСЛЕДУЮЩЕЙ РАБОТЫ И ВНЕСЕНИЯ В DF
    if len(sent_kum) > len(sent_rus):
      razn = len(sent_kum) - len(sent_rus)
      for i in range(razn):
        sent_rus.append('No data')
    elif len(sent_kum) < len(sent_rus):
      diff = len(sent_rus) - len(sent_kum)
      for n in diff:
        sent_kum.append('No data')
    print(f'Без пропусков kum: {len(sent_kum)} ru: {len(sent_rus)}')

    return name, sent_kum, sent_rus, kum, rus


  def create_corpora(self):
    data_list = []
    for tail in range(len(self.name)):
      splitted = self.spliting(self.name[tail], self.text_kum[tail], self.text_ru[tail])
      # print(splitted)
      for i in range(len(splitted[1])):
        row = {'Name': splitted[0], 'Sent_kum': splitted[1][i], 'Sent_rus': splitted[2][i], 'Text_kum': splitted[3], 'Text_rus': splitted[4]}
        data_list.append(row)

    df = pd.DataFrame(data_list)

    return df

In [ ]:
# name[0]

In [ ]:
corp_third= third_corpora(kum, rus, name)
three = corp_third.create_corpora()

Без пропусков kum: 23 ru: 23
Без пропусков kum: 25 ru: 25
Без пропусков kum: 18 ru: 18
Без пропусков kum: 20 ru: 20
Без пропусков kum: 28 ru: 28
Без пропусков kum: 54 ru: 54
Без пропусков kum: 53 ru: 53
Без пропусков kum: 41 ru: 41


In [ ]:
three

,Name,Sent_kum,Sent_rus,Text_kum,Text_rus
0,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,"Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ.,"Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
1,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,Анар пойыны кууныҥ калыгы деп аданар.,Они называли себя лебединым народом.,"Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
2,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,Алдындаҥ пери анаҥ аназы куу полгон деп анар п...,"Они верили, что их праматерью была лебедь.","Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
3,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,"Анар чакшы кӧӧнилиг, улузыраш калык полгон.","Они были гордыми, с хорошими нравами.","Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
4,Куу калыктыҥ чӧрчеги / Сказка о лебедином народе,Коштыйа анаҥ ла чатчын калык куу калыкты тоога...,"Живущий рядом народ считался с ними, они уважа...","Качан, качандаа чер сыртында пир ньаан калык ч...",Давным-давно на земле жил один большой народ. ...
...,...,...,...,...,...
257,Тöреен черим чÿрегимде / Родная земля в сердце,Че чалчылар агач тöзини пьетеҥ артык тезекле ч...,"Скучая по родной земле, Моему сердцу больно.","Кöк ээниг сында, ээс тагларныҥ кийниндеҥ таҥ ч...","На синей широкой гриве, за высокими горами на ..."
258,Тöреен черим чÿрегимде / Родная земля в сердце,"Чÿрек агачтыҥ ньа черге кӱӱни чок та болзо, ки...",No data,"Кöк ээниг сында, ээс тагларныҥ кийниндеҥ таҥ ч...","На синей широкой гриве, за высокими горами на ..."
259,Тöреен черим чÿрегимде / Родная земля в сердце,Тогра черде паркалып Кÿннерими öтиртим.,No data,"Кöк ээниг сында, ээс тагларныҥ кийниндеҥ таҥ ч...","На синей широкой гриве, за высокими горами на ..."
260,Тöреен черим чÿрегимде / Родная земля в сердце,Тöреен черге эригип Меҥ чÿрегим агрыды.,No data,"Кöк ээниг сында, ээс тагларныҥ кийниндеҥ таҥ ч...","На синей широкой гриве, за высокими горами на ..."


In [ ]:
three.to_csv('tails_corpora.csv', index=False)

In [ ]:
def splitting(text_kum: str, text_ru: str, name: str):
  kum = re.sub('\n', ' ', text_kum)
  rus = re.sub('\n', ' ', text_ru)
  sent_kum = list(x.text for x in razdel.sentenize(kum))
  sent_rus = list(x.text for x in razdel.sentenize(rus))


In [ ]:
def spliting(name, tail_kum, tail_ru):
  kum = re.sub('\n', ' ', tail_kum)
  rus = re.sub('\n', ' ', tail_ru)
  sent_kum = list(x.text for x in razdel.sentenize(kum))
  sent_rus = list(x.text for x in razdel.sentenize(rus))
  print(f'С пропусками kum: {len(sent_kum)} ru: {len(sent_rus)}')
  if len(sent_kum) > len(sent_rus):
    razn = len(sent_kum) - len(sent_rus)
    for i in range(razn):
      sent_rus.append('No data')
  elif len(sent_kum) < len(sent_rus):
    diff = len(sent_rus) - len(sent_kum)
    for n in diff:
      sent_kum.append('No data')
  print(f'Без пропусков kum: {len(sent_kum)} ru: {len(sent_rus)}')

  return name, sent_kum, sent_rus

In [ ]:
spliting(name[0], kum[0], rus[0])

In [ ]:
test

('Куу калыктыҥ чӧрчеги / Сказка о лебедином народе',
 ['Качан, качандаа чер сыртында пир ньаан калык чаткан.',
  'Анар пойыны кууныҥ калыгы деп аданар.',
  'Алдындаҥ пери анаҥ аназы куу полгон деп анар пӱткенер.',
  'Анар чакшы кӧӧнилиг, улузыраш калык полгон.',
  'Коштыйа анаҥ ла чатчын калык куу калыкты тоога салган, кӱндӱлен.',
  'Ӧзере садыжып, положып ӧштиглердин ӧзере аданыны алыжып чуртарыны абыр тутып чатканар.',
  'Че по канчеп таа чыллар, шактар ӧте перерде, куу калык пойыныҥ чайалганыны тыҥ улугзыранып, пактанып, тазаҥнап чӱре пердилер.',
  '«- Пис чараш куудаҥ шыккан калык!» - деп, айлантыра чаткан калыкка айтканар.',
  '«- По куш тыҥ ээс кӧк тегриде кудайга чагын ӱчип чӱрчит.',
  'Пистиҥ калык угала даа тыҥ, аҥ учын парчызы писти кӱндӱлерге керек».',
  'Ийде пойыныҥ чайалганыны ӧрӧ кӧдӱрип, аалында чакшы иштенген мелерини кичигден, кичигден уҥдыйа пенер.',
  'Коштыйа чатчын калыктарла ӧштеже пердилер, ӧзере сатыштанды токтатсалдылар, ус иштейтен мелерини кеде эдип, чакшы а